# Repair

In [1]:
from copy import deepcopy
class RepairOperators:
    def __init__(self, solution):
        self.solution = deepcopy(solution)
        self.instance = solution.instance
        self.insertion_log = []  # record

    #*****************************************************************************************************
    #Start of greedy insertion
    def greedy_insertion(self, removed_pairs):
        # loop the removed pairs
        for pickup, delivery in removed_pairs:
            best_cost = float('inf')
            best_route = None
            best_insert_position = None
            # loop each route to find a suitable location to insert 
            for vehicle_id, route in enumerate(self.solution.routes):
                for i in range(1, len(route)):
                    for j in range(i, len(route)):
                        temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                        temp_solution = deepcopy(self.solution)
                        temp_solution.routes[vehicle_id] = temp_route
                        temp_solution.update_all()

                        if temp_solution.is_feasible():
                            cost = temp_solution.objective_function()
                            if cost < best_cost:
                                best_cost = cost
                                best_route = vehicle_id
                                best_insert_position = (i, j)
            
            # update the self.solution
            if best_route is not None and best_insert_position is not None:
                self.insert_single_request(pickup,delivery,best_route, best_insert_position)
        
        return self.solution
    #*****************************************************************************************************
    #End of greedy insertion

    #*****************************************************************************************************
    #Start of regret insertion
    def regret_insertion(self, removed_pairs, k):
        unremoved_pairs = []
        while removed_pairs:
            insertion_costs = []
            for pickup, delivery in removed_pairs: # iterate every pair
                costs = []
                for vehicle_id, route in enumerate(self.solution.routes): # iterate every route
                    min_cost = float('inf')
                    for i in range(1, len(route)):
                        for j in range(i, len(route)):
                            temp_route = route[:i] + [pickup] + route[i:j] + [delivery] + route[j:]
                            temp_solution = deepcopy(self.solution)
                            temp_solution.routes[vehicle_id] = temp_route
                            temp_solution.update_all()

                            if temp_solution.is_feasible():
                                cost = temp_solution.objective_function()
                                if cost < min_cost:
                                    min_cost = cost
                                    best_i, best_j = i,j
                    
                    if min_cost < float('inf'):
                        costs.append((min_cost, vehicle_id, best_i, best_j))
                costs.sort(key=lambda x:x[0])
                insertion_costs.append((pickup, delivery, costs))

            
            best_request = None
            best_route = None
            best_insert_position = None
            for pickup, delivery, costs in insertion_costs:
                # 无法被插入到任何路径，直接跳过
                if len(costs) == 0:
                    removed_pairs.remove((pickup, delivery))
                    unremoved_pairs.append((pickup, delivery))
                    continue
                # 处理插入机会少于k的请求
                if len(costs) > 0 and len(costs) < k:
                    best_request =  (pickup, delivery)
                    best_route = costs[0][1]
                    best_insert_position = (costs[0][2], costs[0][3])
                    break
            
            # 如果没有插入机会少于k的请求，则选择最大遗憾值的请求
            if best_request is None:
                max_regret = float('-inf')
                for pickup, delivery, costs in insertion_costs:
                    regret = sum(cost[0] for cost in costs[:k]) - costs[0][0]
                    if regret > max_regret:
                        max_regret = regret
                        best_request = (pickup, delivery)
                        best_route = costs[0][1]
                        best_insert_position = (costs[0][2], costs[0][3])   

            # 插入最佳请求
            if best_request is not None and best_route is not None and best_insert_position is not None:
                removed_pairs.remove(best_request)
                pickup, delivery = best_request
                self.insert_single_request(pickup, delivery, best_route, best_insert_position)
        
        removed_pairs = unremoved_pairs
        return self.solution
        
    #*****************************************************************************************************
    #End of regret insertion
    def insert_single_request(self, pickup, delivery, vehicle_id, insert_position):
        i, j = insert_position
        self.solution.routes[vehicle_id] = self.solution.routes[vehicle_id][:i] \
                                           + [pickup] + self.solution.routes[vehicle_id][i:j] + [delivery] \
                                           + self.solution.routes[vehicle_id][j:]
        self.solution.update_all() # update all of the things
        self.record_insertion(vehicle_id, pickup, delivery, insert_position)  # 记录插入位置
    
    def record_insertion(self, vehicle_id, pickup, delivery, position):
        """
        记录插入位置
        vehicle_id: 车辆ID
        pickup: 取货点
        delivery: 送货点
        position: 插入位置 (i, j)
        """
        self.insertion_log.append({
        'vehicle_id': vehicle_id,
        'pickup': pickup,
        'delivery': delivery,
        'position': position
        })

    def get_insertion_log(self):
        """
        获取插入日志
        :return: 插入日志
        """
        return self.insertion_log

# test

In [3]:
import random
import numpy as np
from copy import deepcopy
from instance import PDPTWInstance
from solution import PDPTWSolution
from solver import greedy_insertion_init
from operators import RemovalOperators

# 参数设置
n = 10  # pickup点的数量
map_size = 2  # 地图大小
speed = 4  # 车辆速度
extra_time = 10  # delivery点时间窗口起始时间的额外时间
num_vehicles = 5  # 车辆数量
vehicle_capacity = 5  # 车辆容量
battery_capacity = 240  # 电池容量
battery_consume_rate = 1  # 电池消耗率
gamma = 100

instance = PDPTWInstance(n, map_size, speed, extra_time, gamma, seed=1234)
initial_solution = greedy_insertion_init(instance, num_vehicles, vehicle_capacity, battery_capacity, battery_consume_rate)
removal_operators = RemovalOperators(initial_solution)
removed_solution = removal_operators.shaw_removal(num_remove=3,p=3)

In [4]:
removed_solution.unvisited_requests

[(3, 13), (5, 15), (7, 17)]

In [5]:
repair_operators = RepairOperators(removed_solution)
repair_solution = repair_operators.regret_insertion(removed_requests,k=3)
repair_solution.routes

[[0, 7, 6, 17, 16, 8, 4, 14, 18, 9, 19, 0],
 [0, 2, 12, 5, 15, 10, 20, 0],
 [0, 3, 1, 13, 11, 0],
 [0, 0],
 [0, 0]]

In [26]:
repair_operators.insertion_log

[{'vehicle_id': 2, 'pickup': 3, 'delivery': 13, 'position': (1, 2)},
 {'vehicle_id': 1, 'pickup': 5, 'delivery': 15, 'position': (3, 3)},
 {'vehicle_id': 0, 'pickup': 7, 'delivery': 17, 'position': (1, 2)}]

In [6]:
repair_solution.unvisited_requests

[]

# test route

In [20]:
route = [0,0]
for i in range(1, len(route)):
    for j in range(i, len(route)):
        print(i,j)
        print(route[i:] + [4] + route[i:j] + [14] + route[j:])

1 1
[0, 4, 14, 0]
